# MLFlow Intro 

## ML development challanges
<ul>
    <li><h4>Lots of develpment tools</h4></li>
    <li><h4>Hard to track and reproduce results</h4></li>
</ul>

## MLFlow 
<ul>
    <li><h4>Opensource machine learning platform</h4></li>
    <li><h4>Works with any ML library</h4></li>
    <li><h4>Runs the same everywhere</h4></li>
    <li><h4>Allows for collaboration</h4></li>
</ul>

<img src="https://databricks.com/wp-content/uploads/2018/06/mlflow.png">

python -m ipykernel install --user --name myenv --display-name "Python (workshop)

Use mlflow using pip: <br>


In [60]:
#!pip install mlflow

# MLFlow Tracking 

<h3>Key components in tracking</h3>
<ul>
    <li><h4>Parameters - input to code</h4></li>
    <li><h4>Metrics - can change over time</h4></li>
    <li><h4>Artifacts - files, inlcuding models</h4></li>
    <li><h4>Source - what produced the run</h4></li>
</ul>

## imports


In [61]:
import mlflow
import mlflow.sklearn

import numpy as np
import pandas as pd

from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split

Working with remote server:

In [62]:
#!mlflow server -h 0.0.0.0 -p 5050

In [63]:
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [64]:
def build_model(alpha, l1_ratio):
    model = ElasticNet(alpha=alpha, l1_ratio=l1_ratio)
    return model

## Data

In [65]:
df = pd.read_csv("wine-quality.csv")
df.describe()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000,4898.000000
mean,6.854788,0.278241,0.334192,6.391415,0.045772,35.308085,138.360657,0.994027,3.188267,0.489847,10.514267,5.877909
std,0.843868,0.100795,0.121020,5.072058,0.021848,17.007137,42.498065,0.002991,0.151001,0.114126,1.230621,0.885639
min,3.800000,0.080000,0.000000,0.600000,0.009000,2.000000,9.000000,0.987110,2.720000,0.220000,8.000000,3.000000
25%,6.300000,0.210000,0.270000,1.700000,0.036000,23.000000,108.000000,0.991723,3.090000,0.410000,9.500000,5.000000
50%,6.800000,0.260000,0.320000,5.200000,0.043000,34.000000,134.000000,0.993740,3.180000,0.470000,10.400000,6.000000
75%,7.300000,0.320000,0.390000,9.900000,0.050000,46.000000,167.000000,0.996100,3.280000,0.550000,11.400000,6.000000
max,14.200000,1.100000,1.660000,65.800000,0.346000,289.000000,440.000000,1.038980,3.820000,1.080000,14.200000,9.000000


In [66]:
train, test = train_test_split(df)

X_train = train.drop(['quality'], axis=1)
X_test = test.drop(['quality'], axis=1)
y_train = train['quality']
y_test = test['quality']

## Hyperparameters

In [67]:
ALPHA = 0.2
L1_RATIO = 0.6

## Main workflow 

In [68]:
with mlflow.start_run(nested=True):
    model = build_model(ALPHA, L1_RATIO)
    model.fit(X_train, y_train)

    #logging parameters
    mlflow.log_param('alpha', ALPHA)
    mlflow.log_param('l1-ratio', L1_RATIO)


    y_predicted = model.predict(X_test)
    (rmse, mae, r2) = eval_metrics(y_test, y_predicted)

    #logging metrics
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)

    #logging artifacts
    mlflow.log_artifact('wine-quality.csv')

    mlflow.sklearn.log_model(model, "model")

In [69]:
!ls 

Untitled.ipynb     conda.yaml~        mlflow             mlruns
conda.yaml         introduction.ipynb mlflow2            wine-quality.csv



http://localhost:5000

Cleanup

In [70]:
!rm -rf mlruns
!ls

Untitled.ipynb     conda.yaml~        mlflow             wine-quality.csv
conda.yaml         introduction.ipynb mlflow2


# MLFlow Projects

<p>Projects gives us high level format for reproducing runs on different platforms</p>

<b>setup</b>

In [71]:
!git clone https://github.com/greghop/mlflow.git
!ls mlflow

fatal: destination path 'mlflow' already exists and is not an empty directory.
MLproject        conda.yaml       wine-quality.csv
README.md        train.py


<b>MLproject</b>

In [72]:
!cat mlflow/MLproject

name: tutorial

conda_env: conda.yaml

entry_points:
  main:
    parameters:
      alpha: float
      l1_ratio: {type: float, default: 0.1}
    command: "python train.py {alpha} {l1_ratio}"


<b>conda.yaml<b>

In [73]:
!cat mlflow/conda.yaml

name: tutorial
channels:
  - defaults
dependencies:
  - cloudpickle=0.6.1
  - python=3.6
  - numpy=1.14.3
  - pandas=0.22.0
  - scikit-learn=0.19.1
  - pip:
    - mlflow


http://localhost:5000

In [76]:
!python mlflow/train.py 0.8 0.4

Traceback (most recent call last):
  File "mlflow/train.py", line 10, in <module>
    import mlflow
ModuleNotFoundError: No module named 'mlflow'


<b>Running MLflow project</b> <br><br>
mlflow run URI [OPTIONS] <br>

<ul>
<li>-P name=value</li>
<li>-e NAME</li>
</ul>

In [79]:

!mlflow run mlflow/ -P alpha=0.6 -P l1_ratio=0.7

2019/04/10 14:29:57 INFO mlflow.projects: === Created directory /var/folders/nh/p1tcdzns0ns99399dh81m56r0000gn/T/tmpp93piraw for downloading remote URIs passed to arguments of type 'path' ===
2019/04/10 14:29:57 INFO mlflow.projects: === Running command 'source activate mlflow-b93852916f9be8ee2359db52b5dfab5589743459 && python train.py 0.6 0.7' in run with ID 'f119f41166db48369bd85725edfb674d' === 
/Users/dpiskors/anaconda3/envs/mlflow-b93852916f9be8ee2359db52b5dfab5589743459/lib/python3.6/site-packages/mlflow/utils/environment.py:26: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  env = yaml.load(_conda_header)
Elasticnet fantastic 3 model (alpha=0.600000, l1_ratio=0.700000):
  RMSE: 0.8591606388045732
  MAE: 0.648352939480482
  R2: 0.04661433685958705
2019/04/10 14:30:02 INFO mlflow.projects: === Run (ID 'f119f41166db48369bd85725edfb674d') succeeded ===


In [80]:
!mlflow run https://github.com/greghop/mlflow.git -P alpha=0.1 -P l1_ratio=0.5

2019/04/10 14:31:19 INFO mlflow.projects: === Fetching project from https://github.com/greghop/mlflow.git into /var/folders/nh/p1tcdzns0ns99399dh81m56r0000gn/T/tmpgkrhu_x4 ===
2019/04/10 14:31:22 INFO mlflow.projects: === Created directory /var/folders/nh/p1tcdzns0ns99399dh81m56r0000gn/T/tmpn2spbovb for downloading remote URIs passed to arguments of type 'path' ===
2019/04/10 14:31:22 INFO mlflow.projects: === Running command 'source activate mlflow-b93852916f9be8ee2359db52b5dfab5589743459 && python train.py 0.1 0.5' in run with ID 'efc009cf47b74603bd89ef39710ab103' === 
/Users/dpiskors/anaconda3/envs/mlflow-b93852916f9be8ee2359db52b5dfab5589743459/lib/python3.6/site-packages/mlflow/utils/environment.py:26: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  env = yaml.load(_conda_header)
Elasticnet fantastic 3 model (alpha=0.100000, l1_ratio=0.500000):
  RMSE: 0.784501794654

Clean

In [83]:
rm -rf mlruns


# Multistep workflow

In [84]:
!git clone https://github.com/greghop/mlflow2.git
!ls mlflow2

fatal: destination path 'mlflow2' already exists and is not an empty directory.
MLproject        conda.yaml       main.py          wine-quality.csv
README.md        etl.py           train.py


In [85]:
!cat mlflow2/MLproject

name: multistep

conda_env: conda.yaml

entry_points:
  etl:
    parameters:
      scaler: {type: int, default: 1}
    command: "python etl.py --scaler {scaler}"
 
  train:
    parameters:
      run-id: string
      alpha: {type: float, default: 0.1}
      l1-ratio: {type: float, default: 0.1}
    command: "python train.py --run-id {run-id} --alpha {alpha} --l1-ratio {l1-ratio}"

  main: 
    parameters:
      alpha: {type: float, default: 0.1}
      l1-ratio: {type: float, default: 0.1}
    command: "python main.py  --alpha {alpha} --l1-ratio {l1-ratio}"

In [87]:
!mlflow run mlflow2/ -e etl 

2019/04/08 17:32:22 INFO mlflow.projects: === Created directory /var/folders/nh/p1tcdzns0ns99399dh81m56r0000gn/T/tmpfp3d7zyv for downloading remote URIs passed to arguments of type 'path' ===
2019/04/08 17:32:22 INFO mlflow.projects: === Running command 'source activate mlflow-df90610eb3183421bcbf1eef16dc332bc5193c11 && python etl.py --scaler 1' in run with ID 'e492821cb17b405e982250006a1651d7' === 
2019/04/08 17:32:24 INFO mlflow.projects: === Run (ID 'e492821cb17b405e982250006a1651d7') succeeded ===


In [ ]:
!mlflow run mlflow2/ -e etl -P scaler='robust'


Simply go to <h3>http://localhost:5000</h3>